In [16]:
import pandas as pd
import openpyxl as px
import datetime
import calendar

#----------------------
#年月日を入力
#----------------------
invoice_month = input("請求書を作成する年月を入れてください(例：2110)")

#入力された値をdataに変換
dte = datetime.datetime.strptime(invoice_month, "%y%m")
print(dte)
#エラー処理(日付型に変換できないときに表示させたい) 
# try:
#     print(dte)
# except unconverted data remains:
#     print("正しい値を入力してください。")


#選択月の末日を習得
def EOM(dte):
    return dte.replace(day=calendar.monthrange(dte.year, dte.month)[1])

print(EOM(dte))




2021-10-01 00:00:00
2021-10-31 00:00:00


In [22]:
#----------------------
#請求書一覧から顧客CDをDFに取得
#----------------------
customer_df = pd.read_excel("請求一覧創造太郎.xlsx", sheet_name="顧客管理テーブル", index_col=0)
display(customer_df)

,Unnamed: 1
顧客ＣＤ,顧客名
1001,アデックテクノロジー株式会社
1002,株式会社サンワテック
1003,プロト技研株式会社
1004,株式会社AICUBEシステムズ
1005,大川情報開発合同会社
2001,NGCネットワークス株式会社
2002,光栄情報サービス株式会社
3001,株式会社クスダ製作所
3002,大関西リサーチ株式会社


In [11]:
#----------------------
#inputからその月のシートを取得
#----------------------

invoice_month = input("請求書を作成する年月を入れてください(例：2110)")

sheetname = "請求一覧" + invoice_month

month_df = pd.read_excel("請求一覧創造太郎.xlsx", sheet_name=sheetname, header=2)
display(mounth_df)


#相手先コードのユニークを作成
sup_cd = mounth_df["相手先コード"].unique()
print(sup_cd)

,受注No.,相手先名,相手先コード,商品名１,商品名２,検収日,金額（税抜き）,備考
0,210601,NaN,4001,次年度用職員募集,ホームページ作成,10,350000,NaN
1,210602,NaN,2001,FK社名古屋工場向け,ネットワーク設計,16,580000,NaN
2,210701,NaN,1002,SS 社向けプログラム開発,"ss07534, ss07535",20,648000,NaN
3,210702,NaN,1001,TN 社向け営業支援,NaN,20,133000,NaN
4,210803,NaN,4001,次年度用ホームページ,仕様変更対応作業,22,55000,NaN
5,210901,NaN,1001,ＫＹ社向け営業支援,継続活動,25,80000,21年10月分
6,210401,NaN,3001,納品管理システム,変更作業,28,456000,NaN


[4001 2001 1002 1001 3001]
